In [ ]:
import numpy as np 
import pandas as pd
import os

import pandas as pd
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from itertools import combinations


In [ ]:
class CFG:
    data_folder = '../input/jigsaw-toxic-severity-rating/'
    nfolds = 10

# Data

Create all pairs for the Ruddit dataset - we preserve the original scores, so that a difference (pointing which one is more toxic) can be adjusted at modeling time. 

In [ ]:
# ruddit
df = pd.read_csv('../input/ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv')[['comment_id', 'txt', 'offensiveness_score']]
df = df.loc[(df.txt != '[deleted]') & (df.txt != '[removed]')]
df.head(5)

In [ ]:
# create a vector of combinations
id1 = []
id2 = []

for f in combinations(range(df.shape[0]),2):

    id1.append(f[0])
    id2.append(f[1])
    
indices_df = pd.DataFrame(id1, columns=['id1'])
indices_df['id2'] = id2

# shuffle
indices_df = indices_df.sample(frac=1).reset_index(drop=True)

# map to texts and scores
x1 = df.iloc[indices_df.id1][['txt', 'offensiveness_score']].rename(columns={"txt": "txt1", "offensiveness_score": "sc1"}).reset_index(drop=True)
x2 = df.iloc[indices_df.id2][['txt', 'offensiveness_score']].rename(columns={"txt": "txt2", "offensiveness_score": "sc2"}).reset_index(drop=True)

# combine
x3 = pd.concat([x1,x2], axis = 1, ignore_index = True)
x3.columns = ['txt1', 'sc1', 'txt2', 'sc2']

# dump to file
x3.to_csv('ruddit_pairs.csv', index = False)